# 🌊 Titicaca Sentinel - Análisis de Calidad del Agua

Este notebook demuestra cómo usar Google Earth Engine para analizar la calidad del agua del Lago Titicaca usando datos Sentinel-2.

## Índices calculados:
- **NDWI**: Normalized Difference Water Index
- **NDCI**: Normalized Difference Chlorophyll Index
- **CI-green**: Chlorophyll Index Green
- **Turbidez**: Aproximación basada en ratios de bandas
- **Clorofila-a**: Estimación a partir de NDCI

**Autor**: Titicaca Sentinel Project  
**Fecha**: Noviembre 2024

## 1. Setup e Importación de Librerías

In [ ]:
# Importar librerías necesarias
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import sys
import os

# Configurar matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Librerías importadas correctamente")

## 2. Inicializar Google Earth Engine

In [ ]:
# Inicializar Earth Engine
# Si es la primera vez, ejecuta: earthengine authenticate
try:
    ee.Initialize(project='your-gcp-project-id')  # Reemplaza con tu proyecto
    print("✓ Earth Engine inicializado correctamente")
except Exception as e:
    print(f"❌ Error al inicializar Earth Engine: {e}")
    print("Ejecuta: earthengine authenticate")

## 3. Extraer ROI del Lago Titicaca

Usaremos JRC Global Surface Water para obtener la geometría exacta del lago.

In [ ]:
# Definir bounding box del Lago Titicaca
bbox = ee.Geometry.Rectangle([-70.3, -17.3, -68.4, -15.4])

# Cargar JRC Global Surface Water
gsw = ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
occurrence = gsw.select("occurrence")

# Máscara de agua permanente (> 50%)
water_mask = occurrence.gt(50).selfMask()

# Convertir a vectores
vectors = water_mask.reduceToVectors(
    geometry=bbox,
    scale=30,
    geometryType="polygon",
    maxPixels=1e13
)

# Función para agregar área
def add_area(feature):
    area = feature.geometry().area(maxError=10)
    return feature.set("area", area)

# Seleccionar el polígono más grande (el lago)
vectors_with_area = vectors.map(add_area)
lake = vectors_with_area.sort("area", False).first()
roi = lake.geometry()

# Obtener información del área
area_m2 = roi.area(maxError=10).getInfo()
area_km2 = area_m2 / 1e6

print(f"✓ ROI del Lago Titicaca extraído")
print(f"  Área: {area_km2:.2f} km²")
print(f"  Coordenadas centrales: {roi.centroid(maxError=10).coordinates().getInfo()}")

In [ ]:
# Configuración de parámetros
end_date = datetime.now()
start_date = end_date - timedelta(days=6*30)  # 6 meses
cloud_coverage = 20

# Formatear fechas
start_str = start_date.strftime('%Y-%m-%d')
end_str = end_date.strftime('%Y-%m-%d')

print(f"Período de análisis: {start_str} a {end_str}")
print(f"Cobertura de nubes máxima: {cloud_coverage}%")

# Cargar colección Sentinel-2
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(roi)
      .filterDate(start_str, end_str)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_coverage)))

# Contar imágenes
count = s2.size().getInfo()
print(f"\n✓ Imágenes encontradas: {count}")

# Función para enmascarar nubes
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    
    mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0)
            .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0)))
    
    return (image.updateMask(mask)
            .divide(10000)
            .copyProperties(image, ['system:time_start']))

# Aplicar máscara de nubes
s2_masked = s2.map(mask_s2_clouds)

print("✓ Máscara de nubes aplicada")

In [ ]:
# Función para calcular índices
def calculate_indices(image):
    """Calcula índices de calidad del agua"""
    green = image.select('B3')
    red = image.select('B4')
    red_edge1 = image.select('B5')
    nir = image.select('B8')
    
    # NDWI - Normalized Difference Water Index
    ndwi = nir.subtract(green).divide(nir.add(green)).rename('NDWI')
    
    # NDCI - Normalized Difference Chlorophyll Index
    ndci = red_edge1.subtract(red).divide(red_edge1.add(red)).rename('NDCI')
    
    # CI-green - Chlorophyll Index Green
    ci_green = nir.divide(green).subtract(1).rename('CI_green')
    
    # Turbidez - Red/Green ratio
    turbidity = red.divide(green).rename('Turbidity')
    
    # TSM - Total Suspended Matter
    tsm = nir.divide(red).rename('TSM')
    
    # Clorofila-a (aproximación)
    chla = ndci.multiply(50).add(30).clamp(0, 150).rename('Chla_approx')
    
    return image.addBands([ndwi, ndci, ci_green, turbidity, tsm, chla])

# Aplicar cálculo de índices
s2_with_indices = s2_masked.map(calculate_indices)

# Crear composición mediana
composite = s2_with_indices.median().clip(roi)

print("✓ Índices calculados:")
print("  - NDWI (detección de agua)")
print("  - NDCI (clorofila)")
print("  - CI-green (índice verde)")
print("  - Turbidez")
print("  - TSM (materia suspendida)")
print("  - Clorofila-a (aproximada)")

In [ ]:
# Calcular estadísticas para todo el lago
bands = ['NDWI', 'NDCI', 'CI_green', 'Turbidity', 'TSM', 'Chla_approx']

stats = composite.select(bands).reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
    ).combine(
        reducer2=ee.Reducer.percentile([10, 50, 90]),
        sharedInputs=True
    ),
    geometry=roi,
    scale=100,
    maxPixels=1e9
).getInfo()

# Crear DataFrame con estadísticas
stats_df = pd.DataFrame([{
    'Índice': band,
    'Media': stats.get(f'{band}_mean', np.nan),
    'Std Dev': stats.get(f'{band}_stdDev', np.nan),
    'P10': stats.get(f'{band}_p10', np.nan),
    'P50': stats.get(f'{band}_p50', np.nan),
    'P90': stats.get(f'{band}_p90', np.nan)
} for band in bands])

print("✓ Estadísticas del Lago Titicaca:\n")
print(stats_df.to_string(index=False))

In [ ]:
# Visualizar distribución de estadísticas
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Distribución de Índices de Calidad del Agua - Lago Titicaca', 
             fontsize=16, fontweight='bold')

for idx, band in enumerate(bands):
    ax = axes[idx // 3, idx % 3]
    
    # Crear barras con percentiles
    p10 = stats_df.loc[stats_df['Índice'] == band, 'P10'].values[0]
    p50 = stats_df.loc[stats_df['Índice'] == band, 'P50'].values[0]
    p90 = stats_df.loc[stats_df['Índice'] == band, 'P90'].values[0]
    mean = stats_df.loc[stats_df['Índice'] == band, 'Media'].values[0]
    
    positions = [0, 1, 2, 3]
    values = [p10, p50, p90, mean]
    labels = ['P10', 'P50', 'P90', 'Media']
    colors = ['#2ca02c', '#ff7f0e', '#d62728', '#1f77b4']
    
    ax.bar(positions, values, color=colors, alpha=0.7)
    ax.set_xticks(positions)
    ax.set_xticklabels(labels)
    ax.set_title(band, fontweight='bold')
    ax.set_ylabel('Valor')
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Punto de muestreo (centro del lago)
sample_point = roi.centroid(maxError=10)

# Función para extraer valores
def extract_values(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    values = image.select(['NDWI', 'NDCI', 'Turbidity', 'Chla_approx']).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=sample_point,
        scale=30
    )
    return ee.Feature(None, values.set('date', date))

# Extraer serie temporal
time_series = s2_with_indices.map(extract_values)
ts_info = time_series.getInfo()

# Convertir a DataFrame
ts_data = []
for feature in ts_info.get('features', []):
    props = feature['properties']
    ts_data.append(props)

ts_df = pd.DataFrame(ts_data)
ts_df['date'] = pd.to_datetime(ts_df['date'])
ts_df = ts_df.sort_values('date')

print(f"✓ Serie temporal extraída: {len(ts_df)} puntos")
print(f"  Rango: {ts_df['date'].min()} a {ts_df['date'].max()}")
print("\nPrimeras filas:")
print(ts_df.head())

In [ ]:
# Graficar series temporales
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Series Temporales de Índices - Centro del Lago Titicaca', 
             fontsize=16, fontweight='bold')

# NDCI (Clorofila)
axes[0, 0].plot(ts_df['date'], ts_df['NDCI'], marker='o', linestyle='-', 
                color='#2ca02c', linewidth=2)
axes[0, 0].set_title('NDCI (Índice de Clorofila)', fontweight='bold')
axes[0, 0].set_ylabel('NDCI')
axes[0, 0].grid(alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# Clorofila-a
axes[0, 1].plot(ts_df['date'], ts_df['Chla_approx'], marker='s', linestyle='-',
                color='#1f77b4', linewidth=2)
axes[0, 1].set_title('Clorofila-a (aproximada)', fontweight='bold')
axes[0, 1].set_ylabel('Chl-a (μg/L)')
axes[0, 1].grid(alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# Turbidez
axes[1, 0].plot(ts_df['date'], ts_df['Turbidity'], marker='^', linestyle='-',
                color='#ff7f0e', linewidth=2)
axes[1, 0].set_title('Turbidez', fontweight='bold')
axes[1, 0].set_ylabel('Turbidez')
axes[1, 0].set_xlabel('Fecha')
axes[1, 0].grid(alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)

# NDWI
axes[1, 1].plot(ts_df['date'], ts_df['NDWI'], marker='d', linestyle='-',
                color='#9467bd', linewidth=2)
axes[1, 1].set_title('NDWI (Índice de Agua)', fontweight='bold')
axes[1, 1].set_ylabel('NDWI')
axes[1, 1].set_xlabel('Fecha')
axes[1, 1].grid(alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Clasificar riesgo basado en percentiles
ndci_p70 = stats_df.loc[stats_df['Índice'] == 'NDCI', 'P90'].values[0] * 0.7
ndci_p90 = stats_df.loc[stats_df['Índice'] == 'NDCI', 'P90'].values[0]
turb_p70 = stats_df.loc[stats_df['Índice'] == 'Turbidity', 'P90'].values[0] * 0.7
turb_p90 = stats_df.loc[stats_df['Índice'] == 'Turbidity', 'P90'].values[0]

print("Umbrales de clasificación de riesgo:")
print(f"\nNDCI:")
print(f"  P70: {ndci_p70:.4f}")
print(f"  P90: {ndci_p90:.4f}")
print(f"\nTurbidez:")
print(f"  P70: {turb_p70:.4f}")
print(f"  P90: {turb_p90:.4f}")

# Función de clasificación de riesgo
def classify_risk(image):
    ndci = image.select('NDCI')
    turbidity = image.select('Turbidity')
    
    high_risk = ndci.gt(ndci_p90).Or(turbidity.gt(turb_p90))
    medium_risk = (ndci.gt(ndci_p70).Or(turbidity.gt(turb_p70))
                  .And(high_risk.Not()))
    low_risk = high_risk.Not().And(medium_risk.Not())
    
    risk_map = (low_risk.multiply(1)
               .add(medium_risk.multiply(2))
               .add(high_risk.multiply(3))
               .rename('Risk_Level'))
    
    return image.addBands(risk_map)

# Aplicar clasificación
composite_with_risk = classify_risk(composite)

# Calcular distribución de riesgo
risk_stats = composite_with_risk.select('Risk_Level').reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=roi,
    scale=100,
    maxPixels=1e9
).getInfo()

risk_histogram = risk_stats.get('Risk_Level', {})
total_pixels = sum(risk_histogram.values())

print(f"\n✓ Distribución de zonas de riesgo:")
print(f"  🟢 Bajo (1):  {int(risk_histogram.get('1', 0))} píxeles ({risk_histogram.get('1', 0)/total_pixels*100:.1f}%)")
print(f"  🟡 Medio (2): {int(risk_histogram.get('2', 0))} píxeles ({risk_histogram.get('2', 0)/total_pixels*100:.1f}%)")
print(f"  🔴 Alto (3):  {int(risk_histogram.get('3', 0))} píxeles ({risk_histogram.get('3', 0)/total_pixels*100:.1f}%)")

In [ ]:
# Guardar estadísticas
output_dir = '../data/exports'
os.makedirs(output_dir, exist_ok=True)

# Guardar estadísticas a CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
stats_file = f'{output_dir}/lake_statistics_{timestamp}.csv'
stats_df.to_csv(stats_file, index=False)
print(f"✓ Estadísticas guardadas: {stats_file}")

# Guardar serie temporal a CSV
ts_file = f'{output_dir}/time_series_{timestamp}.csv'
ts_df.to_csv(ts_file, index=False)
print(f"✓ Serie temporal guardada: {ts_file}")

# Guardar estadísticas completas a JSON
full_stats = {
    'timestamp': datetime.now().isoformat(),
    'period': {
        'start': start_str,
        'end': end_str
    },
    'lake_area_km2': area_km2,
    'statistics': stats,
    'risk_distribution': {
        'low': int(risk_histogram.get('1', 0)),
        'medium': int(risk_histogram.get('2', 0)),
        'high': int(risk_histogram.get('3', 0))
    },
    'thresholds': {
        'ndci_p70': float(ndci_p70),
        'ndci_p90': float(ndci_p90),
        'turbidity_p70': float(turb_p70),
        'turbidity_p90': float(turb_p90)
    }
}

json_file = f'{output_dir}/analysis_results_{timestamp}.json'
with open(json_file, 'w') as f:
    json.dump(full_stats, f, indent=2)
print(f"✓ Resultados completos guardados: {json_file}")

print("\n✓ Análisis completado exitosamente!")

## 📝 Resumen y Conclusiones

Este notebook ha demostrado:

1. ✅ **Extracción automática del ROI** del Lago Titicaca usando JRC Global Surface Water
2. ✅ **Carga y procesamiento** de imágenes Sentinel-2 con máscaras de nubes
3. ✅ **Cálculo de índices** de calidad del agua (NDWI, NDCI, Turbidez, Clorofila)
4. ✅ **Análisis estadístico** del lago completo con percentiles
5. ✅ **Series temporales** para seguimiento de evolución
6. ✅ **Clasificación de riesgo** automática basada en umbrales relativos
7. ✅ **Exportación de datos** para análisis posteriores

### Próximos pasos sugeridos:

- 🔄 Ejecutar periódicamente para monitoreo continuo
- 📊 Integrar con dashboard web (Streamlit)
- 🤖 Entrenar modelo ML para predicción (Random Forest)
- 🗺️ Generar mapas de calor de riesgo
- 📧 Implementar alertas automáticas

---

**¡Protejamos el Lago Titicaca! 🌊🌍**

## 10. Exportar Datos

## 9. Clasificación de Riesgo Ambiental

## 8. Análisis de Series Temporales

## 7. Visualizar Resultados

## 6. Calcular Estadísticas del Lago

## 5. Calcular Índices de Calidad del Agua

## 4. Cargar y Procesar Datos Sentinel-2